# Integração de Dados Financeiros:

### Extração: 
Extração de dados de várias fontes financeiras, como bancos, cartões de crédito, investimentos.

### Transformação: 
Normalização de moedas, cálculo de saldos consolidados, detecção de transações suspeitas.

### Carregamento: 
Armazenamento dos dados processados em um sistema financeiro para análises e relatórios.

In [113]:
import pandas as pd

In [114]:
data = pd.read_csv('conta.csv')

In [116]:
data.head()

,data,conta,descricao,moeda,valor,horario
0,2023-01-01,Banco A,Salário USD,USD,5000.0,08:00:00
1,2023-01-02,Banco B,Compra no Cartão de Crédito,USD,-100.0,12:30:00
2,2023-01-03,Banco A,Transferência para Investimento,USD,-1500.0,15:45:00
3,2023-01-05,Banco C,Compra no Cartão de Crédito,EUR,-80.0,10:20:00
4,2023-01-10,Banco A,Depósito em Conta,USD,2000.0,09:10:00


In [117]:
data['moeda'].unique()

array(['USD', 'EUR'], dtype=object)

In [118]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   data       44 non-null     object 
 1   conta      44 non-null     object 
 2   descricao  44 non-null     object 
 3   moeda      44 non-null     object 
 4   valor      44 non-null     float64
 5   horario    44 non-null     object 
dtypes: float64(1), object(5)
memory usage: 2.2+ KB


### Arrumando data e hora

In [119]:
data['data'] = pd.to_datetime(data['data'])
data['horario'] = pd.to_datetime(data['horario'], format='%H:%M:%S')
data['horario'] = data['horario'].dt.time
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   data       44 non-null     datetime64[ns]
 1   conta      44 non-null     object        
 2   descricao  44 non-null     object        
 3   moeda      44 non-null     object        
 4   valor      44 non-null     float64       
 5   horario    44 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 2.2+ KB


### Conversão para moeda unica
dolar = 1
euro = 1,05

In [123]:
conversao = 1.05

data['valor'] = pd.to_numeric(data['valor'], errors='coerce')

data['valor_dolar'] = data.apply(lambda row: row['valor'] * conversao if row['moeda'] == 'EUR' else row['valor'], axis=1)

data['moeda'] = 'USD'

del data['valor']

In [124]:
data.head()

,data,conta,descricao,moeda,horario,valor_dolar
0,2023-01-01,Banco A,Salário USD,USD,08:00:00,5000.0
1,2023-01-02,Banco B,Compra no Cartão de Crédito,USD,12:30:00,-100.0
2,2023-01-03,Banco A,Transferência para Investimento,USD,15:45:00,-1500.0
3,2023-01-05,Banco C,Compra no Cartão de Crédito,USD,10:20:00,-80.0
4,2023-01-10,Banco A,Depósito em Conta,USD,09:10:00,2000.0


## Adicionando saldo em conta

In [126]:


data = data.sort_values(['data', 'horario'], ascending=[True, True])

data['saldo_conta'] = 0.0

saldo_acumulado = 0.0
for index, row in data.iterrows():
    saldo_acumulado += row['valor_dolar']
    data.at[index, 'saldo_conta'] = saldo_acumulado
    
data.head()

,data,conta,descricao,moeda,horario,valor_dolar,saldo_conta
0,2023-01-01,Banco A,Salário USD,USD,08:00:00,5000.0,5000.0
1,2023-01-02,Banco B,Compra no Cartão de Crédito,USD,12:30:00,-100.0,4900.0
2,2023-01-03,Banco A,Transferência para Investimento,USD,15:45:00,-1500.0,3400.0
3,2023-01-05,Banco C,Compra no Cartão de Crédito,USD,10:20:00,-80.0,3320.0
4,2023-01-10,Banco A,Depósito em Conta,USD,09:10:00,2000.0,5320.0


### Indentificando transações suspeitas por horario ou valor

In [129]:
data['suspeito_horario'] = ((data['horario'].apply(lambda x: x.hour) >= 23) | (data['horario'].apply(lambda x: x.hour) < 6)).astype(int)


# Calcule a média de retirada
media_retirada = data[data['valor_dolar'] < 0]['valor_dolar'].mean()

# Crie a coluna 'transacao_suspeita' com base nos critérios
data['transacao_suspeita'] = (data['suspeito_horario'] | (data['valor_dolar'] < 0) & (abs(data['valor_dolar']) > 1.5 * abs(media_retirada))).astype(bool)

# Exiba o DataFrame com as novas colunas
data1 = data[['data', 'horario', 'conta', 'descricao', 'moeda', 'valor_dolar', 'saldo_conta', 'transacao_suspeita']]
data1.head()


,data,horario,conta,descricao,moeda,valor_dolar,saldo_conta,transacao_suspeita
0,2023-01-01,08:00:00,Banco A,Salário USD,USD,5000.0,5000.0,False
1,2023-01-02,12:30:00,Banco B,Compra no Cartão de Crédito,USD,-100.0,4900.0,False
2,2023-01-03,15:45:00,Banco A,Transferência para Investimento,USD,-1500.0,3400.0,True
3,2023-01-05,10:20:00,Banco C,Compra no Cartão de Crédito,USD,-80.0,3320.0,False
4,2023-01-10,09:10:00,Banco A,Depósito em Conta,USD,2000.0,5320.0,False


### Renomeando colunas

In [148]:
data1.columns = ['DATA', 'HORA', 'CONTA', 'DESCRIÇÂO', 'MOEDA', 'MOVIMENTAÇÔES', 'SALDO', 'TRANSAÇÂO SUSPEITA']

data1.head()

teste1 = data1

# Salvando no Excel

In [150]:
teste1['DATA'] = pd.to_datetime(teste1['DATA']).dt.strftime('%Y-%m-%d')


teste1.to_excel('output.xlsx', index=False)

teste1 = pd.read_excel('output.xlsx')

